Step 1: Random generations of intensity measure values and failure probabilities through MC simulations

In [1]:
# https://stackoverflow.com/questions/29885371/how-do-i-block-comment-in-jupyter-notebook

In [2]:
# no_values = 5 #for tests
no_values = 512 #this should come from a convergence study


Intentisity measure values for PGA which belongs to 0-1g range

In [3]:
intensity_measure_min = 0 #value derived from the paper
intensity_measure_max = 1.0 #value derived from the paper

Intensity measure values for scour depth which belongs to 0-max scour depth range, with max scour depth > 1.

In [4]:
#A transformation of standard uniform distribution is needed in this case.

In [5]:
crack_starting_scour_depth = 1.00 #m
collapse_scour_depth = 2.54 #m

In [6]:
import numpy as np

In [7]:
# np.random.seed(0) #this numpy function ensures that every time the code is ran, the same random numbers are obtained. Therefore, it must be commented out to produce data for a probability paper.
#https://www.geeksforgeeks.org/generate-random-numbers-from-the-uniform-distribution-using-numpy/#:~:text=To%20generate%20random%20numbers%20from,()%20method%20of%20random%20module.&text=In%20uniform%20distribution%20samples%20are,low%20but%20excludes%20high%20interval.
#https://stackoverflow.com/questions/21494489/what-does-numpy-random-seed0-do
#https://www.tutorialspoint.com/generating-random-number-list-in-python

In [8]:
intensity_measure_values = np.sort(np.random.uniform(size = no_values)) #the sort command is needed otherwise the uniform random numbers don't have an increasing order
#https://sparkbyexamples.com/numpy/numpy-sort-arrays-examples/#:~:text=Array%20sort()-,Use%20numpy.,containing%20elements%20in%20ascending%20order.
#intensity_measure_values = np.random.uniform(size = no_values)) #it gives a vector of random numbers, uniformly distributed, greater or smaller than the following one. -WRONG-

intensity_measure_values_exp = np.expand_dims(intensity_measure_values, 1)

print(type(intensity_measure_values))
print(intensity_measure_values_exp.shape)
print(intensity_measure_values[0:5])

<class 'numpy.ndarray'>
(512, 1)
[0.00027652 0.00189005 0.00293317 0.00441351 0.0044497 ]


In [9]:
transformed_intensity_measure_values = (collapse_scour_depth - crack_starting_scour_depth) * intensity_measure_values + crack_starting_scour_depth

transformed_intensity_measure_values_reshape = np.reshape(transformed_intensity_measure_values, (no_values,1))

print(transformed_intensity_measure_values_reshape.shape)

(512, 1)


In [10]:
failure_probability_values = np.sort(np.random.uniform(size=no_values))
#https://sparkbyexamples.com/numpy/numpy-sort-arrays-examples/#:~:text=Array%20sort()-,Use%20numpy.,containing%20elements%20in%20ascending%20order.
failure_probability_values_exp = np.expand_dims(failure_probability_values, 1)

print(type(intensity_measure_values))
print(failure_probability_values_exp.shape)
print(failure_probability_values_exp[0:5])

<class 'numpy.ndarray'>
(512, 1)
[[0.0036941 ]
 [0.00618508]
 [0.01080154]
 [0.01162867]
 [0.01243106]]


In [11]:
print(type([failure_probability_values]))

<class 'list'>


Importing the fragility curve

In [12]:
#Input data needed are median and std of the fragility curve from the paper.
#These values are referred as deterministic values.
median_fragility = 1.215 #back-calculated from reverting the standard normal distribution, once the logtransformation was applied. It comes from the file median_std_deterministic.ipynb
std_fragility = 0.05 #assumed as the value for the system-level fragility, because the paper does not specify a value. This value derives from fitting bridge damage data. Bridge damage data are obtained from numerical simulations.

In [13]:
from scipy import stats

In [14]:
def lognorm_cdf(x, median, sigma):
    shape = sigma #described by s in the scipy guide
    loc = 0
    scale = median
    return stats.lognorm.cdf(x, shape, loc, scale)

In [15]:
# lognorm_values = lognorm_cdf(intensity_measure_values_exp, median_fragility, std_fragility) #for the PGA case
lognorm_values = lognorm_cdf(transformed_intensity_measure_values, median_fragility, std_fragility) #for the scour depth case

lognorm_values_reshape = np.reshape(lognorm_values, (no_values,1))

print(lognorm_values_reshape.shape)
print(lognorm_values_reshape[0:5])
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)}) #https://stackoverflow.com/questions/22222818/how-to-print-numpy-array-with-3-decimal-places

(512, 1)
[[5.08779026e-05]
 [6.23359956e-05]
 [7.09756708e-05]
 [8.51588576e-05]
 [8.55364308e-05]]


In [16]:
damage_condition=[]
damage_condition = lognorm_values_reshape >= failure_probability_values_exp
#https://thispointer.com/compare-two-numpy-arrays-element-wise-in-python/#compare-two-numpy-arrays-using-operator
#https://www.w3schools.com/python/python_operators.asp
#print(damage_condition.shape)
#print(np.column_stack((lognorm_values_exp[0:5], failure_probability_values_exp[0:5])))


In [17]:
#https://flexiple.com/python/typeerror-only-size-1-array-can-be-converted-to-python-scalars/
damage_condition_boolean = damage_condition.astype(int)
print(damage_condition_boolean[0:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [18]:
#https://stackoverflow.com/questions/60493932/how-to-combine-column-vectors-into-a-matrix
damage_probability_data = np.concatenate([failure_probability_values_exp, lognorm_values_reshape, damage_condition_boolean], axis=1)

print(damage_probability_data[0:5])

[[0.004 0.000 0.000]
 [0.006 0.000 0.000]
 [0.011 0.000 0.000]
 [0.012 0.000 0.000]
 [0.012 0.000 0.000]]


Step 2: Realisations of fragility paramaters

In [19]:
# ai = intensity_measure_values_exp #for the PGA case
ai = transformed_intensity_measure_values_reshape #for the scour depth case

In [20]:
ri = damage_condition_boolean

In [21]:
from scipy.optimize import minimize

In [22]:
# median = 0.5 #test value
# sigma = 0.1 #test value

In [23]:
def MLE_logNorm(parameters):
    median, sigma = parameters

    Fi = lognorm_cdf(ai, median, sigma)
    Li = np.power(Fi, ri) * np.power(1-Fi, 1-ri)

    from math import nan, isnan
    list_with_nan = Li
    list_without_nan = [x for x in list_with_nan if isnan(x)==False]
    
    L_two = np.array(list_without_nan)
    L_three = (L_two[L_two>0])
    #log_L = np.log10(L)
    log_L = np.log10(L_three)

    LL = np.sum(log_L)
    neg_LL = -1*LL

    return neg_LL

In [24]:
from math import inf

In [25]:
#https://stackoverflow.com/questions/57229402/how-to-restrict-optimized-value-to-be-greater-than-0
#https://stackoverflow.com/questions/19244527/scipy-optimize-how-to-restrict-argument-values
con = [{"type": "ineq", "fun": MLE_logNorm}]

#Without specifying bands, negative values of standard deviation were obtained
#The lower bounds assure that the obtained values are greater than zero, while the first upper bound are due to the values of the intensity measure.
bnds = ((0.01, inf), (0.01, inf)) #without specifying bands, negative values of standard deviation were obtained

#Initial guess value
guess = np.array([1.0, 0.1])


In [26]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
#https://stackoverflow.com/questions/30902111/passing-list-of-arguments-of-varying-length-to-scipy-optimize-minimize-understa
#https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

mle_model = minimize(MLE_logNorm, guess, method = 'Nelder-Mead', constraints = con, bounds = bnds, options = {'maxiter':5000}) #assumed no. of iterations

#mle_model = minimize(MLE_logNorm, guess, method = 'Nelder-Mead', bounds = bnds, options = {'maxiter':5000}) #assumed no. of iterations

# mle_model = minimize(MLE_logNorm, guess, method = 'L-BFGS-B', constraints = con, bounds = bnds, options = {'maxiter':5000}) #assumed no. of iterations

#Different results are obtained by applying different minimization methods. 
#[0.99, 0.027] with Nelder-Mead, while [0.99, 0.010] with L-BFGS-B. 
# The listed results sound suspicious because equal to the set bounds. 
#Both the methods cannot handle the specified constraints.

c:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:569: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


In [27]:
mle_model

 final_simplex: (array([[1.135, 0.010],
       [1.136, 0.010],
       [1.136, 0.010]]), array([0.925, 0.925, 0.925]))
           fun: 0.9251637899778014
       message: 'Optimization terminated successfully.'
          nfev: 51
           nit: 28
        status: 0
       success: True
             x: array([1.135, 0.010])

Calculate failure probability with realization values

In [28]:
Fi = lognorm_cdf(ai, mle_model.x[0], mle_model.x[1])
print(Fi[0:5])

[[0.000]
 [0.000]
 [0.000]
 [0.000]
 [0.000]]


In [29]:
stats.lognorm.cdf(ai, mle_model.x[1], loc=0, scale=mle_model.x[0])[0:5]

array([[0.000],
       [0.000],
       [0.000],
       [0.000],
       [0.000]])

In [30]:
investigated_scour_depth = [1.00, 1.20, 1.30, 1.50, 2.00, 2.30, 2.49, 2.54]

In [31]:
Fi = lognorm_cdf(investigated_scour_depth, mle_model.x[0], mle_model.x[1])
print(Fi)

[0.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000]
